# Pulizia "rotten_tomatoes_reviews.csv"

In [1]:
import pandas as pd
import os

In [5]:
# Risali alla cartella "Solutions"
script_dir = os.getcwd()
solutions_dir = os.path.abspath(os.path.join(script_dir, ".."))

# Imposta "Data" come directory di lavoro
data_dir = os.path.join(solutions_dir, "Data")
os.chdir(data_dir)  # Cambia la directory di lavoro

# Ora puoi riferirti direttamente ai file dentro "Data"
file_path = os.path.join("additional_data", "rotten_tomatoes_reviews.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(data_dir, "clean_data")

# Controlla che il file esista prima di caricarlo
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Il file non esiste: {file_path}")

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)

In [6]:
# Rimuove le righe in cui 'review_score', 'publisher_name' e 'critic_name' sono null
df = df.dropna(subset=["review_score", "critic_name", "publisher_name"])

In [7]:
# Funzione per convertire i punteggi
def convert_score(score):
    score = str(score).strip()  # Converti in stringa e rimuovi spazi
    if '/' in score:
        try:
            num, denom = map(float, score.split('/'))
            if denom == 0:
                return None
            score_10 = (num / denom) * 10
            return round(score_10, 1)
        except ValueError:
            return None

    elif score.replace('.', '', 1).isdigit():
        return round(float(score), 1)

    letter_grades = {
        "A+": 10, "A": 9.5, "A-": 9, "B+": 8, "B": 7.5, "B-": 7,
        "C+": 6, "C": 5.5, "C-": 5, "D+": 4, "D": 3.5, "D-": 3,
        "F": 2
    }

    return round(letter_grades.get(score, None), 1) if score in letter_grades else None

# Applica la conversione ai dati
df["review_score"] = df["review_score"].apply(convert_score)

In [8]:
df['review_date'] = pd.to_datetime(df['review_date'])

# Filtraggio delle righe con date a partire dal 1998
df = df[df['review_date'] >= '1998-01-01']

In [9]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "rotten_tomatoes_reviews_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")

# Caricamento file pulito nel database con PostgreSQL

In [10]:
from pymongo import MongoClient

In [11]:
# Configura MongoDB (modifica se necessario)
MONGO_URL = "mongodb://localhost:27017/"
DB_NAME = "ium_database"
COLLECTION_NAME = "rotten_tomatoes_reviews_data"

# Connessione a MongoDB
client = MongoClient(MONGO_URL)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

# Carica il CSV con Pandas
df = pd.read_csv(cleaned_csv_path)

# Converte il DataFrame in JSON-like per MongoDB
data = df.to_dict(orient="records")

# Usa insert_many() per inserire tutto in un'unica operazione (molto più veloce)
collection.insert_many(data, ordered=False); #il ; non fa stampare l'output su jupyter